In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "1" # export OMP_NUM_THREADS=1
os.environ["OPENBLAS_NUM_THREADS"] = "1" # export OPENBLAS_NUM_THREADS=1
os.environ["MKL_NUM_THREADS"] = "1" # export MKL_NUM_THREADS=1
os.environ["VECLIB_MAXIMUM_THREADS"] = "1" # export VECLIB_MAXIMUM_THREADS=1
os.environ["NUMEXPR_NUM_THREADS"] = "1"

In [2]:
import numpy as np
import tensorly as tl
from tensorly.decomposition import matrix_product_state
from scipy.sparse import csr_matrix
from tensorly.tenalg import contract
import torch
from multiprocessing import Pool
import scipy

In [3]:
def core_to_tensor(core, p_vec):
    d = int(len(core)-1)/2
    if d==2:
        result = contract(contract(contract(contract(core[0], 2, core[1], 0), 3, core[2], 0), 3, core[3], 0), 4, core[4], 0)
    elif d==3:
        result = contract(contract(contract(contract(contract(contract(core[0], 2, core[1], 0), 3, core[2], 0), 4, core[3], 0), 4,
                          core[4], 0), 5, core[5], 0), 6, core[6], 0)
    elif d==4:
        result = contract(contract(contract(contract(contract(contract(contract(contract(core[0], 2, core[1], 0), 3, core[2], 0), 4, core[3], 0), 5,
                              core[4], 0), 5, core[5], 0), 6, core[6], 0), 7, core[7], 0), 8, core[8], 0)
    return result.reshape(p_vec, order="F")


In [4]:
def core_to_tensor1(core, p_vec):
    result = contract(contract(contract(contract(contract(core[0], 2, core[1], 0), 3, core[2], 0), 3, core[3], 0), 4,
                          core[4], 0), 5, core[5], 0)
    return result.reshape(p_vec, order="F")

In [5]:
def appproj(A,mtl_rank):
    p_vec = A.shape
    p = np.prod(p_vec)
    d = len(p_vec)
    a0 = 1
    for i in range(d-1):
        a0 = a0 * p_vec[i]
        A = A.reshape(a0,int(p/a0),order="F")
        U,S,V = torch.svd(torch.tensor(A))
        U=U.numpy()
        S=S.numpy()
        V=V.numpy()
        A=U[:,:mtl_rank[i]]@np.diag(S[:mtl_rank[i]])@(V[:,:mtl_rank[i]]).T
        A=A.reshape(p_vec,order="F")
        
    A = A.reshape(p_vec,order="F")
    return A

In [6]:
import time
def PGD(Y, X, trueA, p_vec1, p_vec, mtl_rank, eta, max_iter, tol):
    #p2_vec = size_of_tensor * 2
    p1, N = Y.shape
    p, N = X.shape
    Amat =np.zeros((p1,p))##
    #Amat = trueA
    #A_list = [None] * max_iter
    chg_list = [None] * max_iter
    est_list = [None] * max_iter
    n = 0
    chg = np.inf
    while n < max_iter and chg > tol:
        #print("b")
        start = time.time()
        g_k = Amat - (eta*2/(N-1))*(Amat@X-Y)@np.transpose(X)
        #g_k = g_k.reshape([p,p_vec[2],p_vec[1],p_vec[0]])
        g_k = g_k.reshape(p_vec1+p_vec,order="F")
        #g_k = TTOI(g_k, mtl_rank, iter=10, tol=1e-3)[-1]
        #A_list[n] = g_k
        g_k = appproj(g_k,mtl_rank)
        #g_k = g_k.reshape([p,p_vec[2],p_vec[1],p_vec[0]], order="F")
        g_k = g_k.reshape(p1,p,order="F")
        #g_k = g_k.reshape(p, p, order="F")
        chg = np.linalg.norm(g_k-Amat, "fro")
        chg_list[n] = chg
        est_list[n] = np.linalg.norm(g_k-trueA, "fro")
        if chg>10:
            break
        end=time.time()
        #print(end-start)
        #print(chg)
        #print(np.linalg.norm(g_k-trueA, "fro"))
        #print(n)
        Amat = g_k
        n = n+1

    return ([x for x in chg_list if x is not None],[x for x in est_list if x is not None])

In [7]:
def d1(ind):
    # Step 1 generate G and sigma
    # d=3 p1=p2=p3=10 r=(1,2,2,2,2,2,1)
    # norm(A)=norm(sigma)=5
    # T = [1000,1200,1400,1600,2000]
    p_vec=[3,3,3]
    p = np.prod(p_vec)
    p_vec1=[3,3]
    p1 = np.prod(p_vec1)
    N=600
    scipy.random.seed()
    mtl_rank = [1,1,1,1,1,1]
    global G1; global G2; global G3; global sigma; global G4; global G5; global G6;
    global A_ts; global A_mat
    G1 = np.linalg.svd(np.random.normal(0,1,p_vec1[0]*p_vec1[0]).reshape(p_vec1[0],p_vec1[0]))[0][:,0:mtl_rank[1]].reshape(mtl_rank[0],p_vec1[0],mtl_rank[1],order="F")
    G2 = np.linalg.svd(np.random.normal(0,1,np.square(p_vec1[1]*mtl_rank[1])).reshape(p_vec1[1]*mtl_rank[1],p_vec1[1]*mtl_rank[1]))[0][:,0:mtl_rank[2]].reshape(mtl_rank[1],p_vec1[1],mtl_rank[2],order="F")
    #G3 = np.linalg.svd(np.random.normal(0,1,np.square(p_vec1[2]*mtl_rank[2])).reshape(p_vec1[2]*mtl_rank[2],p_vec1[2]*mtl_rank[2]))[0][:,0:mtl_rank[3]].reshape(mtl_rank[2],p_vec1[2],mtl_rank[3],order="F")
    sigma = np.diag(np.random.normal(0,1,mtl_rank[2]))
    sigma = 5*sigma/np.linalg.norm(sigma)
    G4 = np.linalg.svd(np.random.normal(0,1,np.square(p_vec[0]*mtl_rank[3])).reshape(p_vec[0]*mtl_rank[3],p_vec[0]*mtl_rank[3]))[2][0:mtl_rank[2],:].reshape(mtl_rank[2],p_vec[0],mtl_rank[3],order="F")
    G5 = np.linalg.svd(np.random.normal(0,1,np.square(p_vec[1]*mtl_rank[4])).reshape(p_vec[1]*mtl_rank[4],p_vec[1]*mtl_rank[4]))[2][0:mtl_rank[3],:].reshape(mtl_rank[3],p_vec[1],mtl_rank[4],order="F")
    G6 = np.linalg.svd(np.random.normal(0,1,p_vec[2]*p_vec[2]).reshape(p_vec[2],p_vec[2]))[2][0:mtl_rank[4],:].reshape(mtl_rank[4],p_vec[2],mtl_rank[5],order="F")
    A_ts = core_to_tensor1([G1,G2,sigma,G4,G5,G6], p_vec1+p_vec)
    #A_ts = TTOI(A_ts, [2,2,2,2,2], iter=4, tol=1e-3)[-1]
    #A_mat = A_ts.reshape(p, p, order="F")###
    A_mat = A_ts.reshape(p1, p, order="F")
    #print(np.linalg.norm(A_mat,"fro"))
    #A_mat = A_mat.reshape(p,p)
        
        
        
    #import time
    #start = time.time()
    global X; global Y
    X = torch.empty((p, N), dtype=torch.float64)
    Y = torch.empty((p1, N), dtype=torch.float64)
    for s in range(N):
        X[:,s] = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
        Y[:,s] = torch.tensor(A_mat) @ X[:,s] + torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p1),covariance_matrix=torch.eye(p1)).sample()
    X = X.numpy()
    Y = Y.numpy()


        
    result = PGD(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=[1,1,1,1], eta=0.01, max_iter=2000, tol=1e-4)
    
    return result



In [8]:
d1(1)

([0.10492308603958213,
  0.1027125026507236,
  0.10054809349508809,
  0.09842958468724554,
  0.09635621121603669,
  0.09432707749323617,
  0.09234126074712365,
  0.0903978404444434,
  0.08849590825723105,
  0.08663457183248871,
  0.08481295626286087,
  0.08303020460016258,
  0.08128547793400041,
  0.07957795525958115,
  0.07790683323862369,
  0.07627132590490011,
  0.07467066434122993,
  0.0731040963425955,
  0.0715708860737223,
  0.07007031372593803,
  0.06860167517620427,
  0.06716428165007539,
  0.06575745938967885,
  0.06438054932731646,
  0.06303290676511562,
  0.06171390106087401,
  0.060422915320333385,
  0.05915934609570662,
  0.0579226030906249,
  0.05671210887137008,
  0.05552729858428387,
  0.054367619679306214,
  0.05323253163959297,
  0.05212150571693152,
  0.05103402467306993,
  0.04996958252666876,
  0.04892768430582481,
  0.0479078458060626,
  0.046909593353693275,
  0.045932463574341886,
  0.04497600316662286,
  0.04403976868084134,
  0.04312332630254092,
  0.042226251

In [9]:
Y.shape

(9, 600)

In [10]:
def d2(ind):
    # Step 1 generate G and sigma
    # d=3 p1=p2=p3=10 r=(1,2,2,2,2,2,1)
    # norm(A)=norm(sigma)=5
    # T = [1000,1200,1400,1600,2000]
    p_vec=[4,4,4]
    p = np.prod(p_vec)
    p_vec1=[4,4]
    p1 = np.prod(p_vec1)
    N=600
    scipy.random.seed()
    mtl_rank = [1,1,1,1,1,1]
    global G1; global G2; global G3; global sigma; global G4; global G5; global G6;
    global A_ts; global A_mat
    G1 = np.linalg.svd(np.random.normal(0,1,p_vec1[0]*p_vec1[0]).reshape(p_vec1[0],p_vec1[0]))[0][:,0:mtl_rank[1]].reshape(mtl_rank[0],p_vec1[0],mtl_rank[1],order="F")
    G2 = np.linalg.svd(np.random.normal(0,1,np.square(p_vec1[1]*mtl_rank[1])).reshape(p_vec1[1]*mtl_rank[1],p_vec1[1]*mtl_rank[1]))[0][:,0:mtl_rank[2]].reshape(mtl_rank[1],p_vec1[1],mtl_rank[2],order="F")
    #G3 = np.linalg.svd(np.random.normal(0,1,np.square(p_vec1[2]*mtl_rank[2])).reshape(p_vec1[2]*mtl_rank[2],p_vec1[2]*mtl_rank[2]))[0][:,0:mtl_rank[3]].reshape(mtl_rank[2],p_vec1[2],mtl_rank[3],order="F")
    sigma = np.diag(np.random.normal(0,1,mtl_rank[2]))
    sigma = 5*sigma/np.linalg.norm(sigma)
    G4 = np.linalg.svd(np.random.normal(0,1,np.square(p_vec[0]*mtl_rank[3])).reshape(p_vec[0]*mtl_rank[3],p_vec[0]*mtl_rank[3]))[2][0:mtl_rank[2],:].reshape(mtl_rank[2],p_vec[0],mtl_rank[3],order="F")
    G5 = np.linalg.svd(np.random.normal(0,1,np.square(p_vec[1]*mtl_rank[4])).reshape(p_vec[1]*mtl_rank[4],p_vec[1]*mtl_rank[4]))[2][0:mtl_rank[3],:].reshape(mtl_rank[3],p_vec[1],mtl_rank[4],order="F")
    G6 = np.linalg.svd(np.random.normal(0,1,p_vec[2]*p_vec[2]).reshape(p_vec[2],p_vec[2]))[2][0:mtl_rank[4],:].reshape(mtl_rank[4],p_vec[2],mtl_rank[5],order="F")
    A_ts = core_to_tensor1([G1,G2,sigma,G4,G5,G6], p_vec1+p_vec)
    #A_ts = TTOI(A_ts, [2,2,2,2,2], iter=4, tol=1e-3)[-1]
    #A_mat = A_ts.reshape(p, p, order="F")###
    A_mat = A_ts.reshape(p1, p, order="F")
    #A_mat = A_mat.reshape(p,p)
        
        
        
    #import time
    #start = time.time()
    global X; global Y
    X = torch.empty((p, N), dtype=torch.float64)
    Y = torch.empty((p1, N), dtype=torch.float64)
    for s in range(N):
        X[:,s] = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
        Y[:,s] = torch.tensor(A_mat) @ X[:,s] + torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p1),covariance_matrix=torch.eye(p1)).sample()
    X = X.numpy()
    Y = Y.numpy()


        
    result = PGD(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=[1,1,1,1], eta=0.01, max_iter=2000, tol=1e-4)
    
    return result


In [11]:
def d3(ind):
    # Step 1 generate G and sigma
    # d=3 p1=p2=p3=10 r=(1,2,2,2,2,2,1)
    # norm(A)=norm(sigma)=5
    # T = [1000,1200,1400,1600,2000]
    p_vec=[5,5,5]
    p = np.prod(p_vec)
    p_vec1=[5,5]
    p1 = np.prod(p_vec1)
    N=600
    scipy.random.seed()
    mtl_rank = [1,1,1,1,1,1]
    global G1; global G2; global G3; global sigma; global G4; global G5; global G6;
    global A_ts; global A_mat
    G1 = np.linalg.svd(np.random.normal(0,1,p_vec1[0]*p_vec1[0]).reshape(p_vec1[0],p_vec1[0]))[0][:,0:mtl_rank[1]].reshape(mtl_rank[0],p_vec1[0],mtl_rank[1],order="F")
    G2 = np.linalg.svd(np.random.normal(0,1,np.square(p_vec1[1]*mtl_rank[1])).reshape(p_vec1[1]*mtl_rank[1],p_vec1[1]*mtl_rank[1]))[0][:,0:mtl_rank[2]].reshape(mtl_rank[1],p_vec1[1],mtl_rank[2],order="F")
    #G3 = np.linalg.svd(np.random.normal(0,1,np.square(p_vec1[2]*mtl_rank[2])).reshape(p_vec1[2]*mtl_rank[2],p_vec1[2]*mtl_rank[2]))[0][:,0:mtl_rank[3]].reshape(mtl_rank[2],p_vec1[2],mtl_rank[3],order="F")
    sigma = np.diag(np.random.normal(0,1,mtl_rank[2]))
    sigma = 5*sigma/np.linalg.norm(sigma)
    G4 = np.linalg.svd(np.random.normal(0,1,np.square(p_vec[0]*mtl_rank[3])).reshape(p_vec[0]*mtl_rank[3],p_vec[0]*mtl_rank[3]))[2][0:mtl_rank[2],:].reshape(mtl_rank[2],p_vec[0],mtl_rank[3],order="F")
    G5 = np.linalg.svd(np.random.normal(0,1,np.square(p_vec[1]*mtl_rank[4])).reshape(p_vec[1]*mtl_rank[4],p_vec[1]*mtl_rank[4]))[2][0:mtl_rank[3],:].reshape(mtl_rank[3],p_vec[1],mtl_rank[4],order="F")
    G6 = np.linalg.svd(np.random.normal(0,1,p_vec[2]*p_vec[2]).reshape(p_vec[2],p_vec[2]))[2][0:mtl_rank[4],:].reshape(mtl_rank[4],p_vec[2],mtl_rank[5],order="F")
    A_ts = core_to_tensor1([G1,G2,sigma,G4,G5,G6], p_vec1+p_vec)
    #A_ts = TTOI(A_ts, [2,2,2,2,2], iter=4, tol=1e-3)[-1]
    #A_mat = A_ts.reshape(p, p, order="F")###
    A_mat = A_ts.reshape(p1, p, order="F")
    #A_mat = A_mat.reshape(p,p)
        
        
        
    #import time
    #start = time.time()
    global X; global Y
    X = torch.empty((p, N), dtype=torch.float64)
    Y = torch.empty((p1, N), dtype=torch.float64)
    for s in range(N):
        X[:,s] = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
        Y[:,s] = torch.tensor(A_mat) @ X[:,s] + torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p1),covariance_matrix=torch.eye(p1)).sample()
    X = X.numpy()
    Y = Y.numpy()


        
    result = PGD(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=[1,1,1,1], eta=0.01, max_iter=2000, tol=1e-4)
    
    return result


In [11]:
def d4(ind):
    # Step 1 generate G and sigma
    # d=3 p1=p2=p3=10 r=(1,2,2,2,2,2,1)
    # norm(A)=norm(sigma)=5
    # T = [1000,1200,1400,1600,2000]
    p_vec=[6,6,6]
    p = np.prod(p_vec)
    p_vec1=[6,6]
    p1 = np.prod(p_vec1)
    N=600
    scipy.random.seed()
    mtl_rank = [1,1,1,1,1,1]
    global G1; global G2; global G3; global sigma; global G4; global G5; global G6;
    global A_ts; global A_mat
    G1 = np.linalg.svd(np.random.normal(0,1,p_vec1[0]*p_vec1[0]).reshape(p_vec1[0],p_vec1[0]))[0][:,0:mtl_rank[1]].reshape(mtl_rank[0],p_vec1[0],mtl_rank[1],order="F")
    G2 = np.linalg.svd(np.random.normal(0,1,np.square(p_vec1[1]*mtl_rank[1])).reshape(p_vec1[1]*mtl_rank[1],p_vec1[1]*mtl_rank[1]))[0][:,0:mtl_rank[2]].reshape(mtl_rank[1],p_vec1[1],mtl_rank[2],order="F")
    #G3 = np.linalg.svd(np.random.normal(0,1,np.square(p_vec1[2]*mtl_rank[2])).reshape(p_vec1[2]*mtl_rank[2],p_vec1[2]*mtl_rank[2]))[0][:,0:mtl_rank[3]].reshape(mtl_rank[2],p_vec1[2],mtl_rank[3],order="F")
    sigma = np.diag(np.random.normal(0,1,mtl_rank[2]))
    sigma = 5*sigma/np.linalg.norm(sigma)
    G4 = np.linalg.svd(np.random.normal(0,1,np.square(p_vec[0]*mtl_rank[3])).reshape(p_vec[0]*mtl_rank[3],p_vec[0]*mtl_rank[3]))[2][0:mtl_rank[2],:].reshape(mtl_rank[2],p_vec[0],mtl_rank[3],order="F")
    G5 = np.linalg.svd(np.random.normal(0,1,np.square(p_vec[1]*mtl_rank[4])).reshape(p_vec[1]*mtl_rank[4],p_vec[1]*mtl_rank[4]))[2][0:mtl_rank[3],:].reshape(mtl_rank[3],p_vec[1],mtl_rank[4],order="F")
    G6 = np.linalg.svd(np.random.normal(0,1,p_vec[2]*p_vec[2]).reshape(p_vec[2],p_vec[2]))[2][0:mtl_rank[4],:].reshape(mtl_rank[4],p_vec[2],mtl_rank[5],order="F")
    A_ts = core_to_tensor1([G1,G2,sigma,G4,G5,G6], p_vec1+p_vec)
    #A_ts = TTOI(A_ts, [2,2,2,2,2], iter=4, tol=1e-3)[-1]
    #A_mat = A_ts.reshape(p, p, order="F")###
    A_mat = A_ts.reshape(p1, p, order="F")
    #A_mat = A_mat.reshape(p,p)
        
        
        
    #import time
    #start = time.time()
    global X; global Y
    X = torch.empty((p, N), dtype=torch.float64)
    Y = torch.empty((p1, N), dtype=torch.float64)
    for s in range(N):
        X[:,s] = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
        Y[:,s] = torch.tensor(A_mat) @ X[:,s] + torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p1),covariance_matrix=torch.eye(p1)).sample()
    X = X.numpy()
    Y = Y.numpy()


        
    result = PGD(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=[1,1,1,1], eta=0.01, max_iter=2000, tol=1e-4)
    
    return result


In [13]:
def d5(ind):
    # Step 1 generate G and sigma
    # d=3 p1=p2=p3=10 r=(1,2,2,2,2,2,1)
    # norm(A)=norm(sigma)=5
    # T = [1000,1200,1400,1600,2000]
    p_vec=[8,8,8]
    p = np.prod(p_vec)
    p_vec1=[8,8]
    p1 = np.prod(p_vec1)
    N=600
    scipy.random.seed()
    mtl_rank = [1,1,1,1,1,1]
    global G1; global G2; global G3; global sigma; global G4; global G5; global G6;
    global A_ts; global A_mat
    G1 = np.linalg.svd(np.random.normal(0,1,p_vec1[0]*p_vec1[0]).reshape(p_vec1[0],p_vec1[0]))[0][:,0:mtl_rank[1]].reshape(mtl_rank[0],p_vec1[0],mtl_rank[1],order="F")
    G2 = np.linalg.svd(np.random.normal(0,1,np.square(p_vec1[1]*mtl_rank[1])).reshape(p_vec1[1]*mtl_rank[1],p_vec1[1]*mtl_rank[1]))[0][:,0:mtl_rank[2]].reshape(mtl_rank[1],p_vec1[1],mtl_rank[2],order="F")
    #G3 = np.linalg.svd(np.random.normal(0,1,np.square(p_vec1[2]*mtl_rank[2])).reshape(p_vec1[2]*mtl_rank[2],p_vec1[2]*mtl_rank[2]))[0][:,0:mtl_rank[3]].reshape(mtl_rank[2],p_vec1[2],mtl_rank[3],order="F")
    sigma = np.diag(np.random.normal(0,1,mtl_rank[2]))
    sigma = 5*sigma/np.linalg.norm(sigma)
    G4 = np.linalg.svd(np.random.normal(0,1,np.square(p_vec[0]*mtl_rank[3])).reshape(p_vec[0]*mtl_rank[3],p_vec[0]*mtl_rank[3]))[2][0:mtl_rank[2],:].reshape(mtl_rank[2],p_vec[0],mtl_rank[3],order="F")
    G5 = np.linalg.svd(np.random.normal(0,1,np.square(p_vec[1]*mtl_rank[4])).reshape(p_vec[1]*mtl_rank[4],p_vec[1]*mtl_rank[4]))[2][0:mtl_rank[3],:].reshape(mtl_rank[3],p_vec[1],mtl_rank[4],order="F")
    G6 = np.linalg.svd(np.random.normal(0,1,p_vec[2]*p_vec[2]).reshape(p_vec[2],p_vec[2]))[2][0:mtl_rank[4],:].reshape(mtl_rank[4],p_vec[2],mtl_rank[5],order="F")
    A_ts = core_to_tensor1([G1,G2,sigma,G4,G5,G6], p_vec1+p_vec)
    #A_ts = TTOI(A_ts, [2,2,2,2,2], iter=4, tol=1e-3)[-1]
    #A_mat = A_ts.reshape(p, p, order="F")###
    A_mat = A_ts.reshape(p1, p, order="F")
    #A_mat = A_mat.reshape(p,p)
        
        
        
    #import time
    #start = time.time()
    global X; global Y
    X = torch.empty((p, N), dtype=torch.float64)
    Y = torch.empty((p1, N), dtype=torch.float64)
    for s in range(N):
        X[:,s] = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
        Y[:,s] = torch.tensor(A_mat) @ X[:,s] + torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p1),covariance_matrix=torch.eye(p1)).sample()
    X = X.numpy()
    Y = Y.numpy()


        
    result = PGD(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=[1,1,1,1], eta=0.01, max_iter=2000, tol=1e-4)
    
    return result


In [14]:
def d6(ind):
    # Step 1 generate G and sigma
    # d=3 p1=p2=p3=10 r=(1,2,2,2,2,2,1)
    # norm(A)=norm(sigma)=5
    # T = [1000,1200,1400,1600,2000]
    p_vec=[10,10,10]
    p = np.prod(p_vec)
    p_vec1=[10,10]
    p1 = np.prod(p_vec1)
    N=600
    scipy.random.seed()
    mtl_rank = [1,1,1,1,1,1]
    global G1; global G2; global G3; global sigma; global G4; global G5; global G6;
    global A_ts; global A_mat
    G1 = np.linalg.svd(np.random.normal(0,1,p_vec1[0]*p_vec1[0]).reshape(p_vec1[0],p_vec1[0]))[0][:,0:mtl_rank[1]].reshape(mtl_rank[0],p_vec1[0],mtl_rank[1],order="F")
    G2 = np.linalg.svd(np.random.normal(0,1,np.square(p_vec1[1]*mtl_rank[1])).reshape(p_vec1[1]*mtl_rank[1],p_vec1[1]*mtl_rank[1]))[0][:,0:mtl_rank[2]].reshape(mtl_rank[1],p_vec1[1],mtl_rank[2],order="F")
    #G3 = np.linalg.svd(np.random.normal(0,1,np.square(p_vec1[2]*mtl_rank[2])).reshape(p_vec1[2]*mtl_rank[2],p_vec1[2]*mtl_rank[2]))[0][:,0:mtl_rank[3]].reshape(mtl_rank[2],p_vec1[2],mtl_rank[3],order="F")
    sigma = np.diag(np.random.normal(0,1,mtl_rank[2]))
    sigma = 5*sigma/np.linalg.norm(sigma)
    G4 = np.linalg.svd(np.random.normal(0,1,np.square(p_vec[0]*mtl_rank[3])).reshape(p_vec[0]*mtl_rank[3],p_vec[0]*mtl_rank[3]))[2][0:mtl_rank[2],:].reshape(mtl_rank[2],p_vec[0],mtl_rank[3],order="F")
    G5 = np.linalg.svd(np.random.normal(0,1,np.square(p_vec[1]*mtl_rank[4])).reshape(p_vec[1]*mtl_rank[4],p_vec[1]*mtl_rank[4]))[2][0:mtl_rank[3],:].reshape(mtl_rank[3],p_vec[1],mtl_rank[4],order="F")
    G6 = np.linalg.svd(np.random.normal(0,1,p_vec[2]*p_vec[2]).reshape(p_vec[2],p_vec[2]))[2][0:mtl_rank[4],:].reshape(mtl_rank[4],p_vec[2],mtl_rank[5],order="F")
    A_ts = core_to_tensor1([G1,G2,sigma,G4,G5,G6], p_vec1+p_vec)
    #A_ts = TTOI(A_ts, [2,2,2,2,2], iter=4, tol=1e-3)[-1]
    #A_mat = A_ts.reshape(p, p, order="F")###
    A_mat = A_ts.reshape(p1, p, order="F")
    #A_mat = A_mat.reshape(p,p)
        
        
        
    #import time
    #start = time.time()
    global X; global Y
    X = torch.empty((p, N), dtype=torch.float64)
    Y = torch.empty((p1, N), dtype=torch.float64)
    for s in range(N):
        X[:,s] = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
        Y[:,s] = torch.tensor(A_mat) @ X[:,s] + torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p1),covariance_matrix=torch.eye(p1)).sample()
    X = X.numpy()
    Y = Y.numpy()


        
    result = PGD(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=[1,1,1,1], eta=0.01, max_iter=2000, tol=1e-4)
    
    return result


In [12]:
import pickle
import time
start = time.time()
if __name__ == '__main__':
    l = list(range(300))
    with Pool(25) as p:
        result = p.map(d4,l)
        with open('try.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)

23.61068820953369


In [15]:
import pickle
import time
start = time.time()
if __name__ == '__main__':
    l = list(range(300))
    with Pool(30) as p:
        result = p.map(d1,l)
        with open('p'+str(33)+'N600S5.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)
start = time.time()
if __name__ == '__main__':
    l = list(range(300))
    with Pool(30) as p:
        result = p.map(d2,l)
        with open('p'+str(44)+'N600S5.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)
start = time.time()
if __name__ == '__main__':
    l = list(range(300))
    with Pool(30) as p:
        result = p.map(d3,l)
        with open('p'+str(55)+'N600S5.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)
start = time.time()
if __name__ == '__main__':
    l = list(range(300))
    with Pool(30) as p:
        result = p.map(d4,l)
        with open('p'+str(66)+'N600S5.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)
start = time.time()
if __name__ == '__main__':
    l = list(range(300))
    with Pool(30) as p:
        result = p.map(d5,l)
        with open('p'+str(88)+'N600S5.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)
start = time.time()
if __name__ == '__main__':
    l = list(range(300))
    with Pool(30) as p:
        result = p.map(d6,l)
        with open('p'+str(1010)+'N600S5.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)

4.731523036956787
5.557847499847412
8.754189014434814


KeyboardInterrupt: 

In [13]:
import pickle

with open('try.pkl','rb') as f:
    result = pickle.load(f)

In [14]:
import numpy as np
list1 = []
list2 = []
for i in range(len(result)):
    #print(result[i][0][-1])
    if result[i][0][-1]<0.01 and len(result[i][1])<2000:
        list1.append(result[i][1][-1])
        #print(len(result[i][1]))
        list2.append(len(result[i][1]))
print(np.mean(list1))
print(np.std(list1))

0.2057386755685179
0.027961250830623428


In [15]:
len(list2)

300

In [ ]:
np.max(list2)

In [ ]:
def d1(ind):
    result = 100**3
    return result

In [ ]:
import itertools
import time
from multiprocessing import Pool
        
if __name__ == '__main__':
    l = list(range(900000000))
    with Pool(30) as p:
        result = p.map(d1,l)
        with open('p'+str(1010)+'tt.pkl','wb') as f:
            pickle.dump(result, f)


In [ ]:
from multiprocessing import Pool
def d1(ind):
    return 100**2
if __name__ == '__main__':
    l = list(range(399999))
    with Pool(30) as p:
        result = p.map(d1,l)
        #with open('p'+str(1010)+'N600S5.pkl','wb') as f:
            #pickle.dump(result, f)
